# Demonstration 1: Capturing an OpenMDAO Assembly, editing it, and executing the modified assembly

## Part 1: Capturing the OpenMDAO Assembly

This is the first of three Jupyter Notebooks that show an example of the workflow pictured below:

![Process showing separation of creation, editing, and use of MDAO models](workflow.drawio.svg)

In this notebook engineer 1 is creating an OpenMDAO assembly that covers multiple use cases. In this example we are using the `PrepGeom` group from Aviary. 

<div class="alert alert-block alert-info">
<b>Note:</b> Thanks to Ken Moore from NASA for providing the code to set this example up.
</div>

In [1]:
import openmdao.api as om

from aviary.subsystems.geometry.flops_based.prep_geom import PrepGeom
from aviary.utils.aviary_values import AviaryValues
from aviary.validation_cases.validation_tests import get_flops_data

from aviary.variable_info.functions import override_aviary_vars
from aviary.variable_info.variables import Aircraft

from aviary.validation_cases.validation_data.flops_data.FLOPS_Test_Data import \
    FLOPS_Test_Data
from aviary.variable_info.variables import Aircraft
from standard_evaluator import save_assembly


from aviary.subsystems.geometry.flops_based.prep_geom import PrepGeom

In [2]:
class PreMission(om.Group):

    def initialize(self):
        self.options.declare(
            'aviary_options', types=AviaryValues,
            desc='collection of Aircraft/Mission specific options')

    def setup(self):
        self.add_subsystem('prep_geom', PrepGeom(),
                           promotes=['*'])

    def configure(self):
        aviary_options = self.options['aviary_options']

        override_aviary_vars(self, aviary_options)


case_name = 'LargeSingleAisle1FLOPS'
keys = [
    Aircraft.Fuselage.NUM_FUSELAGES,
    Aircraft.Propulsion.TOTAL_NUM_FUSELAGE_ENGINES,
    Aircraft.VerticalTail.NUM_TAILS,
    Aircraft.Wing.SPAN_EFFICIENCY_REDUCTION,
    Aircraft.Engine.NUM_ENGINES,
    Aircraft.Propulsion.TOTAL_NUM_ENGINES,
]

options = get_flops_data(case_name, preprocess=True, keys=keys)
model_options = {}
for key in keys:
    model_options[key] = options.get_item(key)[0]

prob = om.Problem(PreMission(aviary_options=options))

# Joerg, this is the Model Options feature, that lets us set component options at the top, so that OpenMDAO will
# propagate them to all the components.
prob.model_options['*'] = model_options

prob.setup(force_alloc_complex=True)

flops_data = FLOPS_Test_Data[case_name]
flops_inputs = flops_data['inputs']
flops_outputs = flops_data['outputs']

input_keys = [
        Aircraft.Fuselage.LENGTH,
        Aircraft.Fuselage.MAX_HEIGHT,
        Aircraft.Fuselage.MAX_WIDTH,
        Aircraft.Fuselage.WETTED_AREA_SCALER,

        Aircraft.HorizontalTail.AREA,
        Aircraft.HorizontalTail.ASPECT_RATIO,
        Aircraft.HorizontalTail.TAPER_RATIO,
        Aircraft.HorizontalTail.THICKNESS_TO_CHORD,
        Aircraft.HorizontalTail.VERTICAL_TAIL_FRACTION,
        Aircraft.HorizontalTail.WETTED_AREA_SCALER,

        Aircraft.Nacelle.AVG_DIAMETER,
        Aircraft.Nacelle.AVG_LENGTH,

        Aircraft.VerticalTail.AREA,
        Aircraft.VerticalTail.ASPECT_RATIO,
        Aircraft.VerticalTail.TAPER_RATIO,
        Aircraft.VerticalTail.THICKNESS_TO_CHORD,
        Aircraft.VerticalTail.WETTED_AREA_SCALER,

        Aircraft.Wing.AREA,
        Aircraft.Wing.ASPECT_RATIO,
        Aircraft.Wing.GLOVE_AND_BAT,
        Aircraft.Wing.SPAN,
        Aircraft.Wing.TAPER_RATIO,
        Aircraft.Wing.THICKNESS_TO_CHORD,
        Aircraft.Wing.WETTED_AREA_SCALER,
]

for key in input_keys:
    if key in flops_inputs:
        data = flops_inputs
    else:
        data = flops_outputs
    desired, units = data.get_item(key)
    prob.set_val(key, desired, units)

prob.run_model()


C:\Users\tf715b\Desktop\standard-evaluator\venv\Lib\site-packages\openmdao\core\system.py:2697: PromotionWarning:'prep_geom.canard' <class Canard>: input variable 'aircraft:canard:area', promoted using 'aircraft:canard:area', was already promoted using 'aircraft*'.
C:\Users\tf715b\Desktop\standard-evaluator\venv\Lib\site-packages\openmdao\core\system.py:2697: PromotionWarning:'prep_geom.canard' <class Canard>: input variable 'aircraft:canard:thickness_to_chord', promoted using 'aircraft:canard:thickness_to_chord', was already promoted using 'aircraft*'.
C:\Users\tf715b\Desktop\standard-evaluator\venv\Lib\site-packages\openmdao\core\system.py:2697: PromotionWarning:'prep_geom.canard' <class Canard>: input variable 'aircraft:canard:wetted_area_scaler', promoted using 'aircraft:canard:wetted_area_scaler', was already promoted using 'aircraft*'.
C:\Users\tf715b\Desktop\standard-evaluator\venv\Lib\site-packages\openmdao\core\system.py:2697: PromotionWarning:'prep_geom.characteristic_lengths

Now that we have the OpenMDAO assembly we can show the values of it's inputs. This will be useful to compare with in the other parts of the demo.

In [3]:
_ = prob.model.list_inputs(shape=True)

78 Input(s) in 'model'

varname                                              val              shape  prom_name                                      
---------------------------------------------------  ---------------  -----  -----------------------------------------------
prep_geom
  fuselage_prelim
    aircraft:fuselage:length                         [128.]           (1,)   aircraft:fuselage:length                       
    aircraft:fuselage:max_height                     [13.17]          (1,)   aircraft:fuselage:max_height                   
    aircraft:fuselage:max_width                      [12.33]          (1,)   aircraft:fuselage:max_width                    
  wing_prelim
    aircraft:wing:area                               [1370.]          (1,)   aircraft:wing:area                             
    aircraft:wing:glove_and_bat                      [134.]           (1,)   aircraft:wing:glove_and_bat                    
    aircraft:wing:span                               [117.8

We can also look at the N2 diagram for this model. If we carefully look at this diagram we can see that there are two main components that create outputs from this assembly:

- `characteristic_lengths`
- `total_wetted_area`

We can also see that two components are only using inputs to the group, and are not depending on the preliminary calculations done in other components in this assembly. The two components only depending on inputs to the group are:

- `nacelles`
- `canard`

In [4]:
import os
# Check if running in VS Code
if 'VSCODE_PID' in os.environ:
    display_in_notebook = False
else:
    display_in_notebook = True
om.n2(prob, 'before_NASA.n2.html', display_in_notebook=display_in_notebook,  )

The final step in this part of the workflow we are demonstrating is the engineer storing both the assembly information and the current state of the assembly in a JSON file (`demo_group_nasa.json`) and a HDF5 file (`state.h5`). 

<div class="alert alert-block alert-info">
<b>Note:</b> We use the HDF5 format for storing the state since it is a binary file format which is compact and ensure no loss of accuracy. There are many different viewers and editors available to edit HDF5. A nice viewer is <a href="https://myhdf5.hdfgroup.org/">myHDF5</a>.
</div>

In [5]:
save_assembly(prob, assembly_name='demo_group_nasa.json', state_name='state.h5')